# Experimentation Only

In [1]:
import sys
!{sys.executable} -m pip install tqdm

You are using pip version 9.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [64]:
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
from datetime import datetime
import datetime as dt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

#For Prediction
from sklearn import metrics
from sklearn import preprocessing, cross_validation

import keras
from keras.layers import Dense, Dropout, Input, Conv1D, BatchNormalization, Reshape
from keras.models import Model,Sequential
from keras.datasets import mnist
from tqdm import tqdm
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam, RMSprop

In [3]:
# Read Datasets 
research_data_raw = pd.read_csv('Datasets/bitcoin_research_data_v2.csv')
# Drop unnecessary columns
research_data = research_data_raw.drop('Unnamed: 0', axis=1)
research_data.head()

,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USD,Popularity,Popularity_Normalized
0,2019-05-11,BTCUSD,6337.9,6984.8,6337.9,6793.5,9855.87,66133073.75,12.0,1200.0
1,2019-05-10,BTCUSD,6151.4,6430.0,6110.1,6337.9,9872.86,62166515.28,12.0,1200.0
2,2019-05-09,BTCUSD,5939.6,6174.0,5933.2,6151.4,7405.18,44816872.39,12.0,1200.0
3,2019-05-08,BTCUSD,5744.0,5983.2,5660.0,5939.6,6007.11,35184697.52,12.0,1200.0
4,2019-05-07,BTCUSD,5687.4,5955.8,5687.4,5906.7,3668.12,21496575.65,12.0,1200.0


In [4]:
# Convert data to milliseconds
research_data['Epoch'] = pd.to_datetime(research_data['Date'] + ' 23:59:59.000')
research_data['Epoch'] = (research_data['Epoch'] - dt.datetime(1970,1,1)).dt.total_seconds().astype('int64')
research_data.head()

,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USD,Popularity,Popularity_Normalized,Epoch
0,2019-05-11,BTCUSD,6337.9,6984.8,6337.9,6793.5,9855.87,66133073.75,12.0,1200.0,1557619199
1,2019-05-10,BTCUSD,6151.4,6430.0,6110.1,6337.9,9872.86,62166515.28,12.0,1200.0,1557532799
2,2019-05-09,BTCUSD,5939.6,6174.0,5933.2,6151.4,7405.18,44816872.39,12.0,1200.0,1557446399
3,2019-05-08,BTCUSD,5744.0,5983.2,5660.0,5939.6,6007.11,35184697.52,12.0,1200.0,1557359999
4,2019-05-07,BTCUSD,5687.4,5955.8,5687.4,5906.7,3668.12,21496575.65,12.0,1200.0,1557273599


In [5]:
research_data_imp2 = research_data.copy()
research_data_imp2.sort_values(by='Date', inplace=True, ascending=True) # Sort by date ascending
forecast_out = int(math.ceil(0.0001 * len(research_data_imp2))) # 0.01% of data
#forecast_out = int(math.ceil(0.0025 * len(research_data_imp2))) # 0.05% of data
print(len(research_data_imp2))
print(forecast_out)

1808
1


In [6]:
research_data_imp2['Prediction'] = research_data_imp2['Close'].shift(-forecast_out)
research_data_imp2.tail()

,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USD,Popularity,Popularity_Normalized,Epoch,Prediction
4,2019-05-07,BTCUSD,5687.4,5955.8,5687.4,5906.7,3668.12,21496575.65,12.0,1200.0,1557273599,5939.6
3,2019-05-08,BTCUSD,5744.0,5983.2,5660.0,5939.6,6007.11,35184697.52,12.0,1200.0,1557359999,6151.4
2,2019-05-09,BTCUSD,5939.6,6174.0,5933.2,6151.4,7405.18,44816872.39,12.0,1200.0,1557446399,6337.9
1,2019-05-10,BTCUSD,6151.4,6430.0,6110.1,6337.9,9872.86,62166515.28,12.0,1200.0,1557532799,6793.5
0,2019-05-11,BTCUSD,6337.9,6984.8,6337.9,6793.5,9855.87,66133073.75,12.0,1200.0,1557619199,NaN


In [7]:
def load_data():
    X = np.array(research_data_imp2.drop(['Prediction', 'Symbol', 'Date', 'Popularity'], 1)) # Create X with numeric columns only
    # X = np.array(research_data_imp2['Close'])
    # X = research_data_imp2['Close'].to_frame()
    X = preprocessing.scale(X)
    X_forecast = X[-forecast_out:]
    X = X[:-forecast_out]

    # research_data_imp2.dropna(inplace=True)

    y = np.array(research_data_imp2['Prediction'])
    y = y[:-forecast_out]

    X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.2)
    
    return [X_train, X_test, y_train, y_test]

In [45]:
seq_length = 10

In [46]:
X_train, X_test, y_train, y_test = load_data()
print(X_train.shape)
print(y_train.shape)


ll = X_train.reshape(-1, seq_length, 1)
print(ll.shape)

(1445, 8)
(1445,)
(1156, 10, 1)


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [47]:
# function for getting one mini batch
def get_batch(samples, batch_size, batch_idx):
    start_pos = batch_idx * batch_size
    end_pos = start_pos + batch_size
    return samples[start_pos:end_pos]

In [48]:
learning_rate = 0.001
lr = learning_rate
batch_size = 28
#cond = 0
latent_dim = 20
num_epochs = 50
vis_freq = 2
labels = None 
D_rounds = 3
G_rounds = 1
hidden_units_g = 150
hidden_units_d = 150
num_generated_features = 1

In [83]:
CG = keras.backend.placeholder(shape=[batch_size, seq_length]) #Placeholder 0 (shape: (50,253)) is it 0 or seq_length
CD = keras.backend.placeholder(shape=[batch_size, seq_length]) #Placerholder 1
Z = keras.backend.placeholder(shape=[batch_size, latent_dim])


In [84]:
def sample_Z(batch_size, seq_length, latent_dim):
    sample = np.float32(np.random.normal(size=[batch_size, seq_length, latent_dim]))
    return sample

In [ ]:

def generator(z, c):
    with tf.variable_scope("generator") as scope:
        
        # each step of the generator takes a random seed + the conditional embedding
        repeated_encoding = tf.tile(c,[1, tf.shape(z)[1]])
        repeated_encoding = tf.reshape(repeated_encoding, [tf.shape(z)[0], tf.shape(z)[1],10]) #SHOULD BE 0 INSTEAD OF 253 BUT THE ERROR
        generator_input = tf.concat([repeated_encoding, z], 2)
        
        cell = tf.contrib.rnn.LSTMCell(num_units=hidden_units_g, state_is_tuple=True)
        rnn_outputs, rnn_states = tf.nn.dynamic_rnn(
                cell=cell,
                dtype=tf.float32,
                sequence_length=[seq_length]*batch_size,
                inputs=generator_input)
        rnn_outputs_2d = tf.reshape(rnn_outputs, [-1, hidden_units_g])
        logits_2d = tf.matmul(rnn_outputs_2d, W_out_G) + b_out_G
        output_2d = tf.nn.tanh(logits_2d)
        output_3d = tf.reshape(output_2d, [-1, seq_length, num_generated_features])
    return output_3d


    

In [85]:
G = Sequential()
G.add(keras.backend.tile(CG, [1, keras.backend.shape(Z)[1]]))
G.add(Reshape(target_shape=(keras.backend.shape(Z)[0], keras.backend.shape(Z)[1], 10)))


TypeError: The added layer must be an instance of class Layer. Found: Tensor("Tile_4:0", shape=(28, ?), dtype=float32)

In [82]:
import tensorflow as tf
SS = tf.placeholder(tf.float32, [batch_size, seq_length]) #Placeholder 0 (shape: (50,253)) is it 0 or seq_length
print(SS)
S2 = keras.backend.placeholder(shape=[batch_size, seq_length])
print(S2)
keras.backend.shape(Z)[1]

Tensor("Placeholder_9:0", shape=(28, 10), dtype=float32)
Tensor("Placeholder_10:0", shape=(28, 10), dtype=float32)


<tf.Tensor 'strided_slice_10:0' shape=() dtype=int32>